In [1]:
import json
import pandas as pd
import numpy as np
import os, glob,re

from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

from IPython.core.display import HTML

import torch
DEBUG = None
PATH = '/content/drive/MyDrive/Coleridge_Initiative/input'

## Ideas and research

for BERT not useble very length text, base model 512 tokens

In [ ]:
print(df1.text.apply(lambda x: len(x.split())).describe())
print()
print('All length text in json: ', len(df1.text.str.cat(sep = '\n').split()))

count     18.000000
mean      98.000000
std       95.736157
min       10.000000
25%       27.750000
50%       41.500000
75%      165.000000
max      298.000000
Name: text, dtype: float64

All length text in json:  1764


Nice len in d0fa7568-7d8e-4db9-870f-f9c6f668c17b

min = 10 and max 298 by title, and we check all len(all text) is eq = 1764


Okay lets see all json length param

In [ ]:
def see_length(file: Path):
    # make conf interval for length
    tmp_df = pd.read_json(file)
    tmp_stat = tmp_df.text.apply(lambda x: len(x.split())).describe().to_dict()
    len_text = len(tmp_df.text.str.cat(sep = '\n').split())
    id = file.stem
    return {
        'id':id,
        'min': tmp_stat['min'],
        'mean': tmp_stat['mean'],
        'max': tmp_stat['max'],
        'len_txt': len_text
        }

list_train_json = list(Path(PATH).glob('train/*.json'))

tmp = []
# time by colab is 1h
for file in tqdm(list_train_json):
    tmp_dict = see_length(file)
    tmp.append(
        pd.DataFrame(tmp_dict,index=[0])
        )
length_df = pd.concat(tmp).reset_index(drop=True)
length_df.head()

100%|██████████| 14316/14316 [59:22<00:00,  4.02it/s]


,index,id,min,mean,max,len_txt
0,0,f0249680-2273-4b1a-8a0a-2e8bbf9ec11a,0.0,316.541667,1197.0,15194
1,0,ef4b7964-b7dc-4570-804e-74cc7b804c61,42.0,427.266667,985.0,6409
2,0,ec938f38-37e7-4a35-87bd-e59ffa02d52e,76.0,318.214286,532.0,4455
3,0,f042f6d8-c940-4cda-9b96-75cb95db4ef8,0.0,378.200000,705.0,3782
4,0,ed1a995e-63f9-4542-b515-bcaab0ed4295,0.0,380.789474,2738.0,7235


In [ ]:
length_df.mean(axis = 0)

index         0.000000
min         553.132858
mean        924.293620
max        1864.406259
len_txt    7049.041771
dtype: float64

In [ ]:
length_df['min'].value_counts().sort_values(ascending=False)

0.0         9052
15.0          79
26.0          77
2.0           74
1.0           70
            ... 
328235.0       1
564.0          1
1356.0         1
522.0          1
1014.0         1
Name: min, Length: 632, dtype: int64

In [ ]:
length_df[length_df['min'] < 100].mean(axis = 0)

index         0.000000
min          10.655568
mean        374.762448
max        1352.718951
len_txt    6855.988687
dtype: float64

check where in lable have digits

In [ ]:
def have_digit(txt: str) -> bool:
    return any(char.isdigit() for char in txt)

reusult = df.cleaned_label.apply(lambda x: have_digit(x))

In [ ]:
for i in df[reusult].cleaned_label.values:
    if 'covid' not in i:
        print(i)
        break

characterizing health associated risks and your baseline disease in sars cov 2 charybdis 


In [ ]:
pd.DataFrame(['cas covid 19 antiviral candidate compounds dataset',
              'cas covid 19 antiviral candidate compounds dataset',
              'cas cov antiviral candidate compounds dataset'
])[0].str.contains('covid').sum()

2

find smalles label

In [ ]:
def the_smallest(txt: str)->int:
    tmp = list(len(x) for x in txt.split() if not x.isdigit())
    return tmp[np.argmin(tmp)]

In [ ]:
df['the_smallest'] = df.cleaned_label.apply(lambda x: the_smallest(x))

In [ ]:
df[df.the_smallest == 1].head()

,Id,pub_title,dataset_title,dataset_label,cleaned_label,the_smallest
412,237b2194-9c15-4e33-b9b7-e9a0ab37f499,A Process Oriented Areal Interpolation Techniq...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1
413,c210f9ed-b69e-4bd6-aa60-d94de4efe7e1,Voluntary Restoration: Mitigation's Silent Par...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1
414,a9f6ce1b-829b-4fdf-9c0c-cb04261e21ad,The Multi-Resolution Land Characteristics (MRL...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1
415,bf02931b-7872-4c63-9837-51479e37b5d3,Evaluation of Airborne Lidar Elevation Surface...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1
416,aac0d000-4569-41fe-b071-118d93d6068a,Change in Land Cover along the Lower Columbia ...,Coastal Change Analysis Program,NOAA C-CAP,noaa c cap,1


In [ ]:
def the_smallest_txt_label(txt: str)->int:
    for x in txt.split():
        if (not x.isdigit()) and (len(txt.split()) == 1):
            return len(x)

df['len_label'] =df.cleaned_label.apply(lambda x: the_smallest_txt_label(x))

In [ ]:
df['len_label'].value_counts()

4.0    3673
7.0     280
5.0       2
Name: len_label, dtype: int64

delete in title CHAPTER|Page|Figure

In [ ]:
a = 'LITERATURE REVIEW Page adakdkah Figure'
re.sub(r'(\bCHAPTER|Page|Figure\b)', ' ', a)

'LITERATURE REVIEW   adakdkah  '

## Preprocessing

In [ ]:
def clean_text(txt: str)-> str:
    return re.sub('[^A-Za-z]+', ' ', str(txt).lower()).strip()

def clean_text_add_digit(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

def len_title(txt: str)->str:
    """
    String is p p p Result:  nan
    String is Andi Result:  Andi
    String is pr Result:  nan
    String is aa aa aa Result:  nan
    String is cas cov Result:  nan
    String is cas c Result:  nan
    String is c a s c o v as Result:  nan
    String is a 0 gv) Result:  nan
    String is ft Result:  nan
    String is All all Result:  nan
    String is Alaa 2 Result:  Alaa 2
    String is A Aa2a Result:  A Aa2a
    String is cas Result:  nan
    String is covantiviral Result:  covantiviral
    String is candidate compounds dataset Result:  candidate compounds dataset

    if one word len >= 4 return word
    else 
        if less and one of words >= 4 words
    else nan
    
    """
    if txt == 'nan' or txt == ' ' or txt == '':        
        return txt  

    txt = txt.split()

    # len('Andi'.split()) == 1
    if (len(txt) == 1) and (len(txt[0])>= 4):
        # one word
        txt = txt[0]
    else:
        len_each_word = list(len(i) for i in txt)
        # print(len_each_word, txt)
        max_len_txt = txt[np.argmax(len_each_word)]
        if len(max_len_txt)>= 4:
            if len_each_word == len(txt):
                txt = 'nan'
            else:              
                txt = ' '.join(txt)
        else:
            txt = 'nan'
    return txt

def change_len(txt: str)-> str: 
    if txt == 'nan':
        return txt   
    return ' '.join(list(i for i in txt.split() if len(txt.split()) >= 10))


def clear_data(txt: str, check : bool = False)-> None:
    """
    txt : path to json file
    check: bool if true return dataframe else save json    
    
    """
    # print(txt)
    tmp_df = pd.read_json(txt)
    """
    a = 'CHAPTER 2. LITERATURE REVIEW Page'
    re.sub(r'(\bCHAPTER|Page|Figure\b)', ' ', a) ->   2. LITERATURE REVIEW  
    """
    tmp_df.section_title = tmp_df.section_title.apply(lambda x:
                                                      re.sub(r'(\bCHAPTER|Page|Figure\b)', ' ', str(x)).strip())
    """
    2. delete 
        C. Career activities after obtaining a doctora...
        IV. Changes to the 1998 SED Marital Status
        B. Policy recommendtions for the utilization .
        1A.
        1.
        GH12.
    """

    tmp_df.section_title = tmp_df.section_title.apply(lambda x:
                                                      re.sub('^[A-Z0-9]+(?=\.)', ' ', str(x)).strip())
    """
    if covid in line use clear with digits save else skip digit
    only in covid label have digit
    """
    if (tmp_df.section_title.str.contains('covid', flags=re.IGNORECASE).sum() != 0) or \
       (tmp_df.text.str.contains('covid', flags=re.IGNORECASE).sum() != 0):
        #covid more digits
        tmp_df.section_title = tmp_df.section_title.apply(lambda x: clean_text_add_digit(x))
        tmp_df.text = tmp_df.text.apply(lambda x: clean_text_add_digit(x))
    else:
         # clear digit
        tmp_df.section_title = tmp_df.section_title.apply(lambda x: clean_text(x))
        tmp_df.text = tmp_df.text.apply(lambda x: clean_text(x))

    #chek length title and if not correct droped
    #find blank line
    tmp_df.section_title.replace('', 'nan', inplace=True)
    tmp_df.text.replace('', 'nan', inplace=True)

    # idea for title if sum len words in title == len title drop, [1,1,1] == 'a t u'  
    tmp_df.section_title = tmp_df.section_title.apply(lambda x: len_title(x))
    tmp_df.text = tmp_df.text.apply(lambda x: change_len(x))

    url = '(https?://|w{3}\.)(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    tel = '(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})'

    # tmp_df.dropna(inplace =True)

    for rgx in [url, tel]:
        tmp_df.text = tmp_df.text.apply(lambda x:
                                    re.sub(rgx, ' ', str(x)))
        
    if check is False:
        #save json??        
        path_save = Path(PATH) /'data_train_v2'/ txt.name       
        tmp_df.reset_index(drop=True).to_json(path_save, orient="columns")
    else:
        return tmp_df.reset_index(drop=True)

check works

In [ ]:
f = 'b3498176-8832-4033-aea6-b5ea85ea04c4.json'
f = 'e9057a7a-ea41-4b40-88bf-3bfb8354c807.json'
f = 'f0249680-2273-4b1a-8a0a-2e8bbf9ec11a.json'
f = 'c6e74b14-6da5-43bf-9723-dcce7c2124ff.json'
f = '9e7a78df-4757-40d8-b347-fe6578f0e4ad.json'
f = '755506a3-1516-4c8b-8fc0-ca10027aed61.json'
f = '6bfda128-a0c0-41ba-96a8-ec62731124d9.json'
f = 'e682c0c9-89bf-4bcf-9919-559729dcf714.json'
f = 'ef228135-27af-4278-a225-57acd2ac0a82.json'
# f = 'd9d43c5a-97b8-4520-bf84-bdb7880878ce.json'

f = Path(PATH) /'train' / f
print(clear_data(f, check=True))

                                       section_title                                               text
0                                                nan  most common observed substitutions in sars cov...
1                                               adar  adenosine deamination in dsrnas is attributed ...
2  is there a link between adar1 and autoimmune d...  double stranded rnas are the pathogen associat...
3                         strandedness of sars cov 2  upon entry into the host cell sars cov 2 exist...
4                                            summary  global travel societal interactions and the in...
5                                                     matrix showing the distribution of genomic cha...
6                                                nan  the three types of changes resulting from the ...


In [ ]:
PATH = '/content/drive/MyDrive/Coleridge_Initiative/input'
PATH_SAVE = '/content/drive/MyDrive/Coleridge_Initiative/input/handles_fix'

handles_fix = list(i.name for i in Path(PATH_SAVE).glob('*.*'))
fixed_list_id = pd.read_csv('/content/drive/MyDrive/Coleridge_Initiative/input/fixed_list_id.csv')

script

In [ ]:
for name in tqdm(fixed_list_id.id):
    if name in handles_fix:  
        f = Path(PATH_SAVE) / name 
    else:
        f = Path(PATH) /'train'/ name  
    clear_data(f)  

check correct

In [ ]:
for f in list(Path(PATH).glob('data_train_v2/*.*'))[10:50]:
    tmp = pd.read_json(f)
    print(f)
    print(tmp)

/content/drive/MyDrive/Coleridge_Initiative/input/data_train_v2/59ceea9a-be62-4d3e-82cd-eeba5135a70e.json
   section_title                                               text
0       abstract  abstract background studies investigating the ...
1            nan  objective to determine the diagnostic accuracy...
2            nan  methods to determine the diagnostic accuracy o...
3            nan  and the onset of the pathogenic process as ref...
4            nan  the second data set consisted of alzheimer s d...
5            nan  the first optimistic prior in accordance with ...
6            nan                                                nan
7        results                                                nan
8            nan  figure shows the roc curves for different anal...
9            nan  in particular it shows the curves for the anal...
10           nan  note that the latter were obtained by using th...
11           nan  the roc curves for the logistic regression are...
12        


## AFTER PREPROCESSION

In [20]:
def make_interval(start:int, txt:str, interval: int)->str:
    """
    start: int return re.search() count latter
    txt: str text
    interval: int len text 

    return: str text
    """
    words = txt.split()
    len_interval = interval * 2
    start = len(txt[:start].split())    
    if (start - interval) > 0:
        start = start - interval
        if start + len_interval < len(words):
            words = words[start: start + len_interval]
        else:              
            words = words[start: len(words)]
    else:      
        words = words[0: len_interval]
    return ' '.join(words)

def clean_text_add_digit(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

def make_label(label: list, txt: str, section_title:str)->list:

    new_title =None
    if (section_title == 'nan') or (section_title == np.nan):
        if (txt != 'nan') or (txt != np.nan):
            new_title = re.search(r'\b%s\b' % label[-1], txt)

    if new_title is None:
        out_l = re.search(r'\b%s\b' % label[-1], txt)
        if out_l is None: # in txt no label
            out_l_small = re.search(r'\b%s\b' % label[0], txt)
            if out_l_small is None:
                out_st = re.search(r'\b%s\b' % section_title, txt)                
                if out_st is None:          

                    target = label[-1]
                    start, end = 0, 0
                else:
                    if (section_title == 'nan') or (section_title == np.nan) or \
                       (section_title == ' ')   or   (section_title == ''):
                        target = label[-1]         
                        start, end = 0, 0     
                    else:
                        target = section_title
                        start, end = out_st.start(), out_st.end()            
            else:
                target = label[0]   
                start, end = out_l_small.start(), out_l_small.end() 
        else: 
            target = label[-1]   
            start, end = out_l.start(), out_l.end()
        
        return target, start, end
    else:
        return new_title.group(), new_title.start(), new_title.end()

def main()->None:

    list_json = list(Path(PATH).glob('data_train_v2/*.*'))

    if DEBUG is not None:
        list_json = [np.random.choice(list_json)]

    
    for f in tqdm(list_json):
        tmp = []
        df_head = pd.read_csv(Path(PATH) / 'train.csv')
        labels = df_head[df_head.Id == f.stem].cleaned_label.values
        pub_title = df_head[df_head.Id == f.stem].pub_title.values[0]
        label = sorted(labels)
        with f.open() as json_file:
            data_json = json.load(json_file)
            
            for idx in range(len(data_json['section_title'])):
                txt = data_json['text'][str(idx)]
              
                section_title = data_json['section_title'][str(idx)]
                if txt == 'nan':
                    target, start, end  = np.nan, np.nan, np.nan
                else:
                    target, start, end  = make_label(label, txt, section_title)
                
                # start_txt = 0
                # end_txt = len(txt)
                ori_txt = txt
       
                if len(txt.split()) > 512:
                    if start != end:
                        l = len(txt)
                        interval = 256
                        txt = make_interval(start, txt, interval)

                # fix start ends
                # hmm we save originl tet and len

                fix_len = re.search(r'\b%s\b' % target, txt)
                if fix_len is not None:
                    fix_len_start = fix_len.start()
                    fix_len_end = fix_len.end()
                else:
                    fix_len_start = 0
                    fix_len_end = 0                

                tmp.append(
                    {
                    'question': clean_text_add_digit(pub_title), #National Education Longitudinal Study
                    'text': txt,
                    'answer': target,
                    'answer_start': fix_len_start,
                    'answer_end': fix_len_end,
                    'origin_text': ori_txt,                       
                    'origin_answer_start': start,
                    'origin_answer_end': end,
                    'len text': len(ori_txt.split())                  
                    }
                )       

        if DEBUG is not None:
            tmp_df = pd.DataFrame(tmp)
            tmp_df['id'] = f.stem
            # tmp_df = pd.DataFrame(tmp).dropna(axis = 0) 
            # tmp_df = tmp_df[tmp_df.answer_end != 0].reset_index(drop=True)
            return tmp_df.reset_index(drop=True) 
        else:
            tmp_df = pd.DataFrame(tmp).dropna(axis = 0)
            # end not eq zeros never
            tmp_df['id'] = f.stem        
            tmp_df = tmp_df[tmp_df.answer_end != 0].reset_index(drop=True)
            if tmp_df.shape[0] == 0:
                print('shape = 0', f.stem)         
            else:
                tmp_df.to_csv(Path(PATH) / 'csv_fix_v6' / (f.stem + '.csv'), index = False)

In [21]:
DEBUG=None
main()

  2%|▏         | 245/14002 [00:24<22:11, 10.33it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  4%|▎         | 524/14002 [00:52<23:56,  9.38it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  5%|▍         | 688/14002 [01:08<21:38, 10.25it/s]

shape = 0 137df1c9-9aba-43b6-9714-4baa4189a971


  9%|▉         | 1326/14002 [02:13<21:30,  9.83it/s]

shape = 0 7b0bbe49-2ca9-451c-afab-0517642a263f


 16%|█▌        | 2190/14002 [03:39<19:25, 10.14it/s]

...

 94%|█████████▍| 13154/14002 [21:53<01:24, 10.01it/s]

shape = 0 ccbeae47-bd72-483c-9b00-ae08156c212e


 94%|█████████▍| 13231/14002 [22:01<01:17,  9.99it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 97%|█████████▋| 13605/14002 [22:38<00:37, 10.51it/s]

shape = 0 98958405-bdfc-4ac8-99ea-0873a05f1d80


100%|██████████| 14002/14002 [23:17<00:00, 10.02it/s]


In [ ]:
DEBUG='Fix'
a = main()
a

  0%|          | 0/1 [00:00<?, ?it/s]


,question,text,answer,answer_start,answer_end,origin_text,origin_answer_start,origin_answer_end,len text,id
0,spill in counts in the quantification of 18f f...,background we aim to provide a systematic stud...,alzheimer s disease neuroimaging initiative adni,486,535,background we aim to provide a systematic stud...,486.0,535.0,318,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
1,spill in counts in the quantification of 18f f...,structured myelin basic proteins and as such r...,alzheimer s disease neuroimaging initiative adni,0,0,structured myelin basic proteins and as such r...,0.0,0.0,477,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
2,spill in counts in the quantification of 18f f...,nan,NaN,0,3,nan,NaN,NaN,1,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
3,spill in counts in the quantification of 18f f...,patient data used in the preparation of this a...,alzheimer s disease neuroimaging initiative adni,76,125,patient data used in the preparation of this a...,76.0,125.0,100,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
4,spill in counts in the quantification of 18f f...,pet images were acquired by using dynamic d ac...,adni,174,178,pet images were acquired by using dynamic d ac...,174.0,178.0,181,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
5,spill in counts in the quantification of 18f f...,image processing was performed using the stati...,adni,1380,1384,image processing was performed using the stati...,1380.0,1384.0,370,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
6,spill in counts in the quantification of 18f f...,firstly the variability of wm suvr cgm across ...,adni,167,171,firstly the variability of wm suvr cgm across ...,167.0,171.0,120,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
7,spill in counts in the quantification of 18f f...,the impact of the changes in wm uptake on the ...,adni,347,351,the impact of the changes in wm uptake on the ...,347.0,351.0,242,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
8,spill in counts in the quantification of 18f f...,patient specific activity maps were generated ...,adni,551,555,patient specific activity maps were generated ...,551.0,555.0,219,e9b3ea9c-251e-4b51-8ebf-db7d09325c66
9,spill in counts in the quantification of 18f f...,mc simulations were performed using the open a...,adni,767,771,mc simulations were performed using the open a...,767.0,771.0,205,e9b3ea9c-251e-4b51-8ebf-db7d09325c66


In [ ]:
a.iloc[17]

question               spill in counts in the quantification of 18f f...
text                   if a reference region including wm in our case...
answer                                                              adni
answer_start                                                        1480
answer_end                                                          1484
origin_text            several publications have recently pointed to ...
origin_answer_start                                                 2628
origin_answer_end                                                   2632
len text                                                            1448
id                                  e9b3ea9c-251e-4b51-8ebf-db7d09325c66
Name: 17, dtype: object

In [ ]:
re.search(r'\badni\b', a.iloc[17].text), re.search(r'\badni\b', a.iloc[17].origin_text)

(<re.Match object; span=(1480, 1484), match='adni'>,
 <re.Match object; span=(2628, 2632), match='adni'>)

Check correct

In [22]:
zero_df = [
        '137df1c9-9aba-43b6-9714-4baa4189a971',
        '7b0bbe49-2ca9-451c-afab-0517642a263f',
        '65ec7b24-bf8b-4e4c-b34e-7992de18773d',
        '9ebf1f28-9b6e-45fc-81b4-413cf6850c84',
        'e0b6860c-e898-421d-bf29-f3873878c07d',
        'd6b68a94-2c61-4a38-8e81-7bc524c54ebb',
        '20a69766-063f-4dd7-9e4f-1d44b571bdb8',
        '526e90f3-4b69-4466-85a5-d1528495d541',
        'bc24201a-3698-4720-acfb-0eb5c7dc5f25',
        'ff1bce8b-d207-4d5f-9a67-50b59bc7b975',
        '3fbea7e1-eee3-41db-bb2c-15bf1367fb70',
        '2c014ca6-04b7-4672-b594-e5aaf23cf2d5',
        '703ed31d-8fbd-42a7-9bdd-797abf0adaa0',
        '30ac418c-77f3-47d5-9a1a-ccacafa26c50',
        '5ba41bf5-1f88-45b0-8138-e435202cb361',
        'ded210ea-c8cd-4873-ba82-d114695687fb',
        '26b9745c-e700-4732-b1eb-3e1713a13f57',
        'e573ab99-54f7-4bac-bfd9-dd29430600f5',
        'd675107e-6640-4831-8e55-b5f90e6410bc',
        'ae3defe7-13e5-46db-84a6-4dc299918750',
        '4ff22cdc-741d-4276-8480-c7e94d7f84f0',
        'c6b1ae20-4299-4167-a1db-a4e37f5a872c',
        '90a2aaf0-8596-4830-a138-a44a3226d5c7',
        'd237ff71-b2d2-4a70-a014-f08b32b0cddd',
        'c27bdedd-b717-492c-a48b-d8d07677c1b5',
        '88ec1a9d-213c-4d66-8a9f-226671b9e13a',
        'ec9e0a3c-9415-4fc5-bd55-b823fb3f5115',
        '6ee08022-c96c-4f62-8c8a-2b1ab2a9bd2a',
        '0280af2e-f57b-4cde-98b9-0becd5b44c95',
        'ac6f1a97-c13a-4058-8ded-4ddd6534c66d',
        'bf84fe87-0d9b-48ab-a24e-ccccfb60af2a',
        '550ea55c-6e6d-4dbb-8736-fb497c4234d9',
        '9cd31f6a-f900-4388-bbaa-91c0ac781565',
        'cb21f8af-8296-4970-ad64-24821f2eeb61',
        '99096154-27e5-4b1a-9622-2ba0b1fcfcab',
        'ccbeae47-bd72-483c-9b00-ae08156c212e',
        '98958405-bdfc-4ac8-99ea-0873a05f1d80'
        ]

In [24]:
assert len(list(Path(PATH).glob('csv_fix_v6/*.*'))) == 14002 - len(zero_df), 'error saved not eq'
a = pd.read_csv(np.random.choice(list(Path(PATH).glob('csv_fix_v6/*.*'))))
a

,question,text,answer,answer_start,answer_end,origin_text,origin_answer_start,origin_answer_end,len text,id
0,ensemble merit merge feature selection for enh...,bagging follows a bootstrap method of data sel...,bagging,0,7,bagging follows a bootstrap method of data sel...,0.0,7.0,416,09336949-3842-4d47-b110-23e93d5bb48d
1,ensemble merit merge feature selection for enh...,na ve bayes classifier is a statistical techni...,na ve bayes,0,11,na ve bayes classifier is a statistical techni...,0.0,11.0,70,09336949-3842-4d47-b110-23e93d5bb48d
2,ensemble merit merge feature selection for enh...,there are phases in the proposed merit merge f...,ensemble feature selection,1599,1625,there are phases in the proposed merit merge f...,1599.0,1625.0,616,09336949-3842-4d47-b110-23e93d5bb48d
3,ensemble merit merge feature selection for enh...,the results are evaluated based on the perform...,results,4,11,the results are evaluated based on the perform...,4.0,11.0,464,09336949-3842-4d47-b110-23e93d5bb48d
4,ensemble merit merge feature selection for enh...,t r sivapriya is the registered user of adni d...,alzheimer s disease neuroimaging initiative adni,107,156,t r sivapriya is the registered user of adni d...,107.0,156.0,86,09336949-3842-4d47-b110-23e93d5bb48d


## Concat all csv

In [25]:
# len 13964
# how maked
dfs = []
for f in tqdm(list(Path(PATH).glob('csv_fix_v6/*.*'))):
    dfs.append(pd.read_csv(f))

df = pd.concat(dfs)
df = df.dropna()# fix 4
df = df.reset_index(drop=True)
df.to_csv('/content/drive/MyDrive/Coleridge_Initiative/input/v6_data_qa.csv', index = False)
df.head(), df.shape

100%|██████████| 13965/13965 [01:01<00:00, 225.56it/s]


(                                            question  ...                                    id
 0  the impact of evaluation use on accounting pro...  ...  40bf999d-8537-47f7-8913-bd67b060146d
 1  the impact of evaluation use on accounting pro...  ...  40bf999d-8537-47f7-8913-bd67b060146d
 2  the sars cov 2 spike glycoprotein biosynthesis...  ...  5631ebad-191f-44d7-9c21-89b4f84852dc
 3  mechanisms of pacific summer water variability...  ...  43f2034e-9220-4103-84bf-79f67b75f29c
 4  mechanisms of pacific summer water variability...  ...  43f2034e-9220-4103-84bf-79f67b75f29c
 
 [5 rows x 10 columns], (62543, 10))

## To fix issure 2

Okay i find problem i not change start and end after cut text old param not correct

In [ ]:
from termcolor import colored
import textwrap

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Coleridge_Initiative/input/v3_data_qa.csv')
print(f'Size data: {df.shape[0]}, count uniques: {len(df.question.unique())}')

Size data: 62586, count uniques: 13902


In [ ]:
df.iloc[9999]

question               seeking explanation in theory reflections on t...
text                   in transition and the principal investigator r...
answer                                                   data collection
answer_start                                                         256
answer_end                                                           271
origin_text            in may and june a cover letter that described ...
origin_answer_start                                                  624
origin_answer_end                                                    639
len text                                                            4203
id                                  2d858fc7-3c56-47de-94fc-b156c53afb55
Name: 9999, dtype: object

In [ ]:
ans = df.iloc[9999].answer
txt = df.iloc[9999].text
re.search(r'\b%s\b' % ans, txt)

<re.Match object; span=(256, 271), match='data collection'>

In [ ]:
print('Question: ', df.iloc[9999].question)
print('Answer: ', df.iloc[9999].answer)
print('Correct')
print(colored(txt[:256], 'white'))
print(colored(txt[256:271+1], 'green'))
print(colored(txt[271+1:], 'white'))

Question:  seeking explanation in theory reflections on the social practices of organizations that distribute public use microdata files for research purposes
Answer:  data collection
Correct
in transition and the principal investigator reported that no one was available to answer questions about data confidentiality however we were able to gather limited descriptive information about this microdata set through their web site by the end of the 
data collection 
period four additional data sets had been identified and information on a total of microdata sets that met the study criteria had been gathered these longitudinal microdata files contain detailed portraits of individuals families and househ


# to fix 3 len count latter but i need words

In [ ]:
def make_interval(start:int, txt:str, interval: int)->str:
    """
    start: int return re.search() count latter
    txt: str text
    interval: int len text 

    return: str text
    """
    words = txt.split()
    len_interval = interval * 2
    start = len(txt[:start].split())    
    if (start - interval) > 0:
        start = start - interval
        print(f'start {start}')
        print('if in')
        if start + len_interval < len(words):       
            print(f'if in in {start + len_interval}')
            words = words[start: start + len_interval]
        else:
            print(f'if else {start + len_interval}')      
            words = words[start: len(words)]
    else:
        print(f'else in {start - interval}')
        words = words[0: len_interval]
    return ' '.join(words)

txt = """
farm finance theme overview are the good times really over</s> the sector level trends reflect the economic performance of the farm economy as a whole however changes in sector level income and total sector real estate asset values accrue to farm operators as well as other agricultural stakeholders including nonoperator landowners according to a new usda survey the tenure ownership and transition of agricultural land total survey of land in farms in the contiguous states was rented or leased a portion of land in farms of this rented land was rented from other farm operators however the majority of land in farms of rented land was rented from non operators therefore both farm operators and non operators benefit from or are vulnerable to changes in farm income and asset values non operator landowners rented out of all land in farms which represented of the sector s value of real estate assets according to the total survey figure in contrast non operator landowners hold a disproportionately low portion of sector debt only as a result non operator landowners account for of farm real estate equity the farm sector s balance sheet nests the balance sheet of both operators and non operator landlords because non operators own a larger share of the sector s real estate assets than real estate debt the farm sector s balance sheet may appear relatively stronger than if only the assets and debt owned by farm operations are examined in addition certain farms may use debt more aggressively than others causing the sector balance sheet to mask heterogeneity among farms therefore looking at the leverage for farm operations and in particular farm businesses may better illustrate the extent sector level trends are reflected in the financial standing of individual farm operations farm businesses those farms with at least in sales or farms with lower revenues where the operator s primary occupation is farming account for more than of the sector s production and hold of all farm assets and of farm debt according to the agricultural resource management survey arms therefore examining changes in the leverage position of farm businesses will provide a good gauge of how closely changes in the solvency risk for these operations compares to the total sector s solvency risk which includes non operator landlords in addition to farm operators because farm businesses represent such a substantial portion of the farm sector the change in the average farm business d a ratios shows a similar pattern to that seen in the sector as a whole after reaching a near record low of in the average farm business d a ratio has been increasing and is expected to be higher at in</s>
farm finance theme overview are the good times really over</s> the sector level trends reflect the economic performance of the farm economy as a whole however changes in sector level income and total sector real estate asset values accrue to farm operators as well as other agricultural stakeholders including nonoperator landowners according to a new usda survey the tenure ownership and transition of agricultural land total survey of land in farms in the contiguous states was rented or leased a portion of land in farms of this rented land was rented from other farm operators however the majority of land in farms of rented land was rented from non operators therefore both farm operators and non operators benefit from or are vulnerable to changes in farm income and asset values non operator landowners rented out of all land in farms which represented of the sector s value of real estate assets according to the total survey figure in contrast non operator landowners hold a disproportionately low portion of sector debt only as a result non operator landowners account for of farm real estate equity the farm sector s balance sheet nests the balance sheet of both operators and non operator landlords because non operators own a larger share of the sector s real estate assets than real estate debt the farm sector s balance sheet may appear relatively stronger than if only the assets and debt owned by farm operations are examined in addition certain farms may use debt more aggressively than others causing the sector balance sheet to mask heterogeneity among farms therefore looking at the leverage for farm operations and in particular farm businesses may better illustrate the extent sector level trends are reflected in the financial standing of individual farm operations farm businesses those farms with at least in sales or farms with lower revenues where the operator s primary occupation is farming account for more than of the sector s production and hold of all farm assets and of farm debt according to the agricultural resource management survey arms therefore examining changes in the leverage position of farm businesses will provide a good gauge of how closely changes in the solvency risk for these operations compares to the total sector s solvency risk which includes non operator landlords in addition to farm operators because farm businesses represent such a substantial portion of the farm sector the change in the average farm business d a ratios shows a similar pattern to that seen in the sector as a whole after reaching a near record low of in the average farm business d a ratio has been increasing and is expected to be higher at in</s>
"""
print(len(txt.split()), len(txt))

896 5361


In [ ]:
make_interval(5218, txt, 256)

start 612
if in
if else 1124


'disproportionately low portion of sector debt only as a result non operator landowners account for of farm real estate equity the farm sector s balance sheet nests the balance sheet of both operators and non operator landlords because non operators own a larger share of the sector s real estate assets than real estate debt the farm sector s balance sheet may appear relatively stronger than if only the assets and debt owned by farm operations are examined in addition certain farms may use debt more aggressively than others causing the sector balance sheet to mask heterogeneity among farms therefore looking at the leverage for farm operations and in particular farm businesses may better illustrate the extent sector level trends are reflected in the financial standing of individual farm operations farm businesses those farms with at least in sales or farms with lower revenues where the operator s primary occupation is farming account for more than of the sector s production and hold of a

In [ ]:
896-612

284

In [ ]:
a = '''disproportionately low portion of sector debt only as a result non operator landowners account for of farm real estate equity the farm sector s balance sheet nests the balance sheet of both operators and non operator landlords because non operators own a larger share of the sector s real estate assets than real estate debt the farm sector s balance sheet may appear relatively stronger than if only the assets and debt owned by farm operations are examined in addition certain farms may use debt more aggressively than others causing the sector balance sheet to mask heterogeneity among farms therefore looking at the leverage for farm operations and in particular farm businesses may better illustrate the extent sector level trends are reflected in the financial standing of individual farm operations farm businesses those farms with at least in sales or farms with lower revenues where the operator s primary occupation is farming account for more than of the sector s production and hold of all farm assets and of farm debt according to the agricultural resource management survey arms therefore examining changes in the leverage position of farm businesses will provide a good gauge of how closely changes in the solvency risk for these operations compares to the total sector s solvency risk which includes non operator landlords in addition to farm operators because farm businesses represent such a substantial portion of the farm sector the change in the average farm business d a ratios shows a similar pattern to that seen in the sector as a whole after reaching a near record low of in the average farm business d a ratio has been increasing and is expected to be higher at in</s>'''

len(a.split())

284

## to fix 4 answer have nan

because label not in text but in text have nan
skip

In [6]:
df = pd.read_csv('/content/drive/MyDrive/Coleridge_Initiative/input/v4_data_qa.csv')
df_head = pd.read_csv(os.path.join(PATH, 'train.csv'))

In [4]:
df_isna = df[df.answer.isna()]
df_isna

,question,text,answer,answer_start,answer_end,origin_text,origin_answer_start,origin_answer_end,len text,id
1046,state of the climate in 2009,july as m to m occurred in northern caucasia o...,NaN,1930,1933,july as m to m occurred in northern caucasia o...,1930.0,1933.0,367,d60fdd8e-cc74-46f5-ad7e-6ddb47e4ab85
3294,regime shift of the south china sea sst in the...,to the west pacific ocean in the northern scs ...,NaN,1457,1460,the pdo or inter decadal pacific oscillation i...,3664.0,3667.0,946,41353837-186e-4814-ba1c-63ecb841cc42
16001,state of the climate in 2019,ii precipitation precipitation over the sahel ...,NaN,1407,1410,ii precipitation precipitation over the sahel ...,1407.0,1410.0,429,11703fa1-62d0-46ea-9b99-160b5dc81dad
18678,structured sparse canonical correlation analys...,the difference numbers in parentheses between ...,NaN,153,156,the difference numbers in parentheses between ...,153.0,156.0,421,e3643a65-5682-4c23-94ac-203bfceef7d1
23176,base year sample design report national educat...,selection of students parents teachers and sch...,NaN,1115,1118,selection of students parents teachers and sch...,1115.0,1118.0,372,89facd61-149d-42a6-a720-3d1175f499fd
27645,interannual coral 14c records of surface water...,c records used in this study blue circle houbi...,NaN,1171,1174,c records used in this study blue circle houbi...,1171.0,1174.0,2603,66698158-8404-4094-a8c3-e023a041388b
33240,the effect of the use of mathematics course in...,matemati i kullanma dersi al ma grubunda yer a...,NaN,709,712,matemati i kullanma dersi al ma grubunda yer a...,709.0,712.0,575,95d21d82-310e-4ce7-9e5a-30bbdec1eb8d


In [7]:
df_head[df_head.Id == '66698158-8404-4094-a8c3-e023a041388b']

,Id,pub_title,dataset_title,dataset_label,cleaned_label
1480,66698158-8404-4094-a8c3-e023a041388b,Interannual coral Δ14C records of surface wate...,World Ocean Database,National Oceanic and Atmospheric Administratio...,national oceanic and atmospheric administratio...
11710,66698158-8404-4094-a8c3-e023a041388b,Interannual coral Δ14C records of surface wate...,World Ocean Database,World Ocean Database,world ocean database


In [ ]:
df_isna.iloc[5].origin_text

'c records used in this study blue circle houbihu taiwan and red circle palaui philippines this study gray diamond guam usa andrews et al light green diamond palau glynn et al yellow diamond ishigaki island japan orange triangle langkai indonesia fallon guilderson light blue triangle lombok strait indonesia guilderson et al green square currimao philippines hirabayashi yokoyama suzuki miyairi aze siringan maeda pink square hon tre vietnam and brown square con dao vietnam mitsuguchi et al the red line across the strait at e running from n to n define the transect where surface zonal currents and sea surface heights from the simple ocean data assimilation data set soda v http apdrc soest hawaii edu dods public data soda soda pop are derived shown in figures and c respectively nec north equatorial current necc north equatorial counter current kc kuroshio current ki kuroshio intrusion mc mindanao current scs south china sea wpo western pacific ocean itf indonesian throughflow from to sv an

In [8]:
df.shape

(62587, 10)

In [11]:
df = df.dropna()
df[df.answer.isna()]

,question,text,answer,answer_start,answer_end,origin_text,origin_answer_start,origin_answer_end,len text,id


## to fix 5 

label is " "

In [ ]:
def make_label(label: list, txt: str, section_title:str)->list:

    new_title =None
    if (section_title == 'nan') or (section_title == np.nan):
        if (txt != 'nan') or (txt != np.nan):
            new_title = re.search(r'\b%s\b' % label[-1], txt)

    if new_title is None:
        out_l = re.search(r'\b%s\b' % label[-1], txt)
        if out_l is None: # in txt no label
            out_l_small = re.search(r'\b%s\b' % label[0], txt)
            if out_l_small is None:
                out_st = re.search(r'\b%s\b' % section_title, txt)                
                if out_st is None:          

                    target = label[-1]
                    start, end = 0, 0
                else:
                    """
                    fix 5
                    problem if title is nan or ' '
                    we find this in text its bad
                    """
                    if (section_title == 'nan') or (section_title == np.nan) or (section_title == ' '):
                        target = label[-1]         
                        start, end = 0, 0     
                    else:
                        target = section_title
                        start, end = out_st.start(), out_st.end()            
            else:
                target = label[0]   
                start, end = out_l_small.start(), out_l_small.end() 
        else: 
            target = label[-1]   
            start, end = out_l.start(), out_l.end()
        
        return target, start, end
    else:
        return new_title.group(), new_title.start(), new_title.end()